# Модель защиты персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
# просмотр, где находится каталог с файлами на COLAB
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# получение доступа к каталогу и уточнение названия папок
import os
os.listdir('/content/drive/My Drive/Colab Notebooks/Яндекс/Проект 8 Защита персональных данных клиентов/ДАННЫЕ')

In [ ]:
# загрузка данных
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Яндекс/Проект 8 Защита персональных данных клиентов/ДАННЫЕ/insurance.csv')

In [ ]:
#df = pd.read_csv('/datasets/insurance.csv')

In [ ]:
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


## Вывод

**В результате знакомства с данными, установлено:**

1. Пропуски отсутствуют.
2. Тип данных int64, кроме столбцов "Возраст" и "Зарплата" - float64.
3. Размер датафрейма (5000, 5).
4. Данные готовы к моделированию. 

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1}  P^T X^T y
$$
$$
w' = P^{-1} (P^T)^{-1} P^T w
$$
$$
w' = P^{-1} (P P^{-1})^T  w
$$
$$
w' = P^{-1} E^T w
$$
$$
w' = P^{-1} w
$$

### Анализ вопроса изменится ли качество линейной регрессии, если признаки умножить на обратимую матрицу

In [ ]:
features = df.drop(columns=['Страховые выплаты'])
target = df['Страховые выплаты']

In [ ]:
features.shape

(5000, 4)

In [ ]:
target.shape

(5000,)

In [ ]:
# деление данных на обучающую и валидационную выборки в соотношении 75:25
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [ ]:
features_train.shape

(3750, 4)

In [ ]:
features_valid.shape

(1250, 4)

In [ ]:
target_train.shape

(3750,)

In [ ]:
target_valid.shape

(1250,)

In [ ]:
# расчет R2 для LinearRegression
model = LinearRegression()
model.fit(features_train, target_train)
y_true = target_valid
y_pred = pd.DataFrame(model.predict(features_valid))

result = r2_score(y_true, y_pred)

print("Качество линейной регрессии (R2):", round(result, 3))

Качество линейной регрессии (R2): 0.435


In [ ]:
features_befor_inv = np.array(features)

In [ ]:
features_befor_inv

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

In [ ]:
# рандомная матрица
rand_matrix = np.random.normal(size=(4, 4))
rand_matrix 

array([[-1.58861682, -0.03130764, -0.5864376 ,  1.18361501],
       [-1.00985699, -1.14793663, -0.54464088, -0.23910045],
       [-0.48708197, -0.85238038,  1.04252979,  0.96399518],
       [-0.59965534, -0.41731185,  1.1072089 , -0.5871269 ]])

In [ ]:
# матрица обратная рандомной
matrix_inv = np.linalg.inv(rand_matrix)
matrix_inv

array([[-0.50995451, -0.00403907,  0.29197509, -0.54700524],
       [ 0.43066114, -0.65747579, -0.47658853,  0.35343415],
       [-0.03087164, -0.35548784,  0.24363438,  0.48255248],
       [ 0.15651744, -0.19894305,  0.49998693, -0.48574039]])

In [ ]:
# единичная матрица, получаемая перемножением рандомной на ее обратную
ons_matrix = matrix_inv.dot(rand_matrix)
ons_matrix

array([[ 1.00000000e+00, -2.97343091e-18,  4.23472916e-17,
        -3.32863741e-18],
       [ 3.45280160e-16,  1.00000000e+00,  5.58270125e-17,
        -1.56845286e-16],
       [-3.10824201e-17, -2.87910912e-19,  1.00000000e+00,
        -3.70247818e-17],
       [-1.03520236e-16, -5.34262571e-17,  3.42943309e-18,
         1.00000000e+00]])

In [ ]:
# перемножение признаков на обратную матрицу (от рандомной)
features_after_inv = features_befor_inv.dot(matrix_inv)
features_after_inv.shape

(5000, 4)

In [ ]:
features_after_inv

array([[ -1513.92952096, -17659.35655416,  12065.51722447,
         23948.06109341],
       [ -1153.15527268, -13538.98090305,   9236.68346506,
         18352.76649413],
       [  -635.81520205,  -7484.31152281,   5102.50097371,
         10143.85168339],
       ...,
       [ -1037.62223347, -12064.58531509,   8250.67378378,
         16364.62629536],
       [ -1000.06838399, -11639.51783592,   7958.15130925,
         15785.23744128],
       [ -1241.68338359, -14451.41877251,   9879.00342918,
         19600.49412387]])

In [ ]:
# деление данных на обучающую и валидационную выборки в соотношении 75:25
features_train, features_valid, target_train, target_valid = train_test_split(
    features_after_inv, target, test_size=0.25, random_state=12345)

In [ ]:
# расчет R2 для LinearRegression
model = LinearRegression()
model.fit(features_train, target_train)
y_true = target_valid
y_pred = pd.DataFrame(model.predict(features_valid))

result = r2_score(y_true, y_pred)

print("Качество линейной регрессии (R2):", round(result, 3))

Качество линейной регрессии (R2): 0.435


### Вывод:

**В результате перемножения матриц, установлено:**

1. Умножение признаков на обратную матрицу не дает потери качества линейной регрессии (R2=0.435=0.435).
2. Параметры линейной регрессии в исходной задаче и в преобразованной одинаковы. Изменению были подвергнуты лишь признаки. В исходной задаче они были в первозданном виде, а в преобразованной задаче признаки подвергли условно кодированию. В результате чего, уже невозможно отличить какой столбец отвечал за возраст, какой за количество детей и т.д.

## Алгоритм преобразования

In [ ]:
# функция возвращает рандомную матрицу на основе введенных данных:
# - число "сдвига" при шифровании
# - исходный ключ шифрования (строка)
def cipher_caesar():
  print("Число сдвига при шифровании:")
  k=int(input())
  print("Исходный ключ:")
  s=input()
  l=len(s)
  str=' ,?;:-\)(*!.'
  string = []
  for i in range(l):
    if 1040<=ord(s[i])<=1071-k:
      string.append(chr(ord(s[i])+k))
    elif 1071-k+1<=ord(s[i])<=1071:
      string.append(chr(1039+(ord(s[i])-(1071-k))))
    elif 1072<=ord(s[i])<=1103-k:
      string.append(chr(ord(s[i])+k))
    elif 1103-k+1<=ord(s[i])<=1103:
      string.append(chr(1071+(ord(s[i])-(1103-k))))

    elif 65<=ord(s[i])<=90-k:
      string.append(chr(ord(s[i])+k))
    elif 90-k+1<=ord(s[i])<=90:
      string.append(chr(64+(ord(s[i])-(90-k))))
    elif 97<=ord(s[i])<=122-k:
      string.append(chr(ord(s[i])+k))
    elif 122-k+1<=ord(s[i])<=122:
      string.append(chr(96+(ord(s[i])-(122-k))))
    elif s[i] in str:
      string.append(s[i])
  print("Зашифрованный ключ =>", string)
  s = string
  rand_state = []
  summ = 0
  for i in range(len(s)):
    rand_state.append(ord(s[i]))
  for j in range(len(rand_state)):
    summ = summ + int(rand_state[j])
  zn = f'{summ}'
  num_loc = summ ** (1 / len(zn))
  num_scale = summ ** (1 / summ)
  print("Параметр loc:", num_loc)
  print("Параметр scale:", num_scale)
  rand_matrix = np.random.normal(loc=num_loc, scale=num_scale, size=(4, 4))
  #print("Рандомная матрица с параметрами на основе зашифрованного ключа:")
  return rand_matrix


In [ ]:
# применение функции cipher_caesar()
matrix = cipher_caesar()

In [ ]:
# рандомная матрица полученная шифрованием
matrix

In [ ]:
# матрица обратная рандомной
matrix_inv = np.linalg.inv(matrix)
matrix_inv

array([[-8.77188389e-01, -3.17022222e+00,  9.07926911e-01,
         3.02621361e+00],
       [-3.80851479e-01,  3.25052827e-04, -2.84528792e-01,
         7.15090192e-01],
       [ 1.81272982e-01,  6.61193692e-01,  4.16000698e-01,
        -1.21878356e+00],
       [ 1.18355357e+00,  2.72498141e+00, -1.10188918e+00,
        -2.65687049e+00]])

**Алгоритм**

1. Ввод числа сдвига при шифровании
2. Ввод исходного ключа (строка с символами на английском или русском языке)
3. Шифрование исходного ключа по методу "Цезаря" (https://ru.wikipedia.org/wiki/%D0%A8%D0%B8%D1%84%D1%80_%D0%A6%D0%B5%D0%B7%D0%B0%D1%80%D1%8F) и получение на этой основе параметров loc & scal
4. Построение рандомной матрицы (4, 4) с параметрами loc & scal

**Обоснование**

   1. *Данный способ защиты персональных данных основан на перемножении признаков на обратную матрицу. Отсутствие потери качества (R2) линейной регрессии подтверждено в предыдущих расчетах. Однако, в дополнение усиления защиты в общий алгоритм включен метод шифрования "Цезаря" формирующий, на основе числа "сдвига" и "исходного ключа", параметры loc & scal для построения рандомной матрицы (4, 4).*
   2. *На основе предложенного способа защиты персональных данных можно зашифровать информацию и предоставить ее в общий доступ. При этом, когда любой желающий попытается получить указанные данные он должен будет ввести свой "исходный ключ" и число "сдвига" при этом данные будут опять подвергнуты шифрованию, что позволит производить их постоянную перекодировку без потери качества (R2) линейной регресии.*

## Проверка алгоритма

In [ ]:
# матрица обратная рандомной
matrix_inv = np.linalg.inv(matrix)
matrix_inv

In [ ]:
# перемножение признаков на обратную матрицу (от рандомной)
features_dot_inv = features_befor_inv.dot(matrix_inv)
features_dot_inv.shape

(5000, 4)

In [ ]:
# деление данных на обучающую и валидационную выборки в соотношении 75:25
features_train, features_valid, target_train, target_valid = train_test_split(
    features_after_inv, target, test_size=0.25, random_state=12345)

In [ ]:
# расчет R2 для LinearRegression
model = LinearRegression()
model.fit(features_train, target_train)
y_true = target_valid
y_pred = pd.DataFrame(model.predict(features_valid))

result = r2_score(y_true, y_pred)

print("Качество линейной регрессии (R2):", round(result, 3))

Качество линейной регрессии (R2): 0.435


## Общий вывод

**I. В результате знакомства с данными, установлено:**

1. Пропуски отсутствуют.
2. Тип данных int64, кроме столбцов "Возраст" и "Зарплата" - float64.
3. Размер датафрейма (5000, 5).
4. Данные готовы к моделированию. 

**II. В результате перемножения матриц, установлено:**

1. Умножение признаков на обратную матрицу не дает потери качества линейной регрессии (R2=0.435=0.435).
2. Параметры линейной регрессии в исходной задаче и в преобразованной одинаковы. Изменению были подвергнуты лишь признаки. В исходной задаче они были в первозданном виде, а в преобразованной задаче признаки подвергли условно кодированию. В результате чего, уже невозможно отличить какой столбец отвечал за возраст, какой за количество детей и т.д.

**III. Предложен, следующий алгоритм защиты персональных данных:**

1. Ввод числа сдвига при шифровании
2. Ввод исходного ключа (строка с символами на английском или русском языке)
3. Шифрование исходного ключа по методу "Цезаря" (https://ru.wikipedia.org/wiki/%D0%A8%D0%B8%D1%84%D1%80_%D0%A6%D0%B5%D0%B7%D0%B0%D1%80%D1%8F) и получение на этой основе параметров loc & scal
4. Построение рандомной матрицы (4, 4) с параметрами loc & scal

**IV. В результате проверки качества предсказания модели линейной регрессии установлено, что качество не ухудшилось (R2=0.435)**
